In [5]:
#############################################
####################Ohio#####################
#############################################
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime

#Reding old resutls and making backup
oldresult=pd.read_csv('COVIDcases_v3.csv')
oldresult.to_csv('COVIDcases_v3.csv.bk')

result=pd.DataFrame()

#opening page and getting html info
page=requests.get('https://coronavirus.ohio.gov/wps/portal/gov/covid-19/home')
soup=bs(page.content,'html.parser')

#Searching for cards with info
statscards=soup.findAll("div",{"class":'stats-cards__item'})

#Looping through each card and extracting useful information for each
for statcard in statscards:
    num_=statcard.find('div',{'class':'stats-cards__number'})
    num=num_.get_text().split('\n')[1].split(' ')
    try:
        num=float(num[len(num)-1].replace(',',''))
    except:
        continue
    label=statcard.find('div',{'class':'stats-cards__label'}).get_text().split('\n')[1].split('  ')
    label=label[len(label)-1]
    result.loc[0,label]=num
    
# Checking to see if ro already exists in my database
check_=(result.loc[0,:]==oldresult.drop(columns=['Days','Date'])).any()
check=(check_==False).any()
        
        
if check == True:
    #adding a day to the tally
    result.loc[0,'Days']=oldresult['Days'].index[-1]+1
    d=datetime.datetime.today()
    result.loc[0,'Date']=str(d.month)+'-'+str(d.day)+'-'+str(d.year)
    newresult=pd.concat([oldresult,result],axis=0)
    newresult.reset_index(drop=True,inplace=True)
    newresult.to_csv('COVIDcases_v3.csv',index=False)
else:
    print('Already have this dataset')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:44: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [6]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import bokeh.plotting as plt
from bokeh.layouts import column
from bokeh.io import output_notebook, output_file
from math import pi
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum

#read in data
################################
dset=pd.read_csv('COVIDcases_v3.csv')


#Fitting Data
################################

#making exponential fit
expfit=np.polyfit(dset.Days[4:],np.log(dset.loc[4:,'Confirmed Cases']),1)
x_expfit=np.linspace(0,dset.Days.index[-1]+3,dset.Days.index[-1]+3+1)
y_expfit=np.exp(expfit[1])*np.exp(expfit[0]*x_expfit)

# Linear fit last three points
linfit=np.polyfit(dset.Days[-3:],dset.loc[len(dset)-3:,'Confirmed Cases'],1)
xlin=x_expfit
ylin=linfit[0]*xlin+linfit[1]

#####################################
############plotting data############
#####################################


#Plot 1 data trends
#####################################

#outfile

#setting up figure
TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
]

p=plt.figure(plot_width=700, plot_height=400, y_range=(0,25000),
             background_fill_color="beige",tooltips=TOOLTIPS)

#plotting data points
p.circle(dset.Days,dset['Confirmed Cases'],size=20, color="navy",alpha=0.5,legend_label='COVID19 Cases')
p.circle(dset.Days,dset['Number of Hospitalizations in Ohio'],size=10,color='green',legend_label='Hospitalization')
p.circle(dset.Days,dset['Total Deaths'],size=10,color='red',legend_label='Deaths')
#Plotting exponential fit
#p.line(x_expfit,y_expfit,line_width=1, color='black',legend_label='Exp Fit')

#plotting last three point linear fit
p.line(xlin,ylin,line_width=1,color='green',legend_label='Last 3 lin Fit')

p.yaxis.axis_label="Confirmed Infected Count"
p.xaxis.axis_label="Number of Days"
p.title.text="Total Infected Count vs Days:    Prediction Tomorrow: "+str(ylin[-3])
p.title.align="center"
p.legend.location="top_left"

slope=np.diff(dset['Confirmed Cases'])/np.diff(dset.Days)

p2=plt.figure(plot_width=700, plot_height=400, background_fill_color="beige",tooltips=TOOLTIPS)
p2.circle(dset.Days[1:],slope,size=20, color='navy')
p2.yaxis.axis_label="Rate of Infection (diff(infect count)/diff(days))"
p2.xaxis.axis_label="Number of Days"
p2.title.text="Total Addition Infected by Day"
p2.title.align="center"

#####################################
############pie graph################
#####################################
total=dset.loc[len(dset)-1,'Confirmed Cases']
hosp=dset.loc[len(dset)-1,'Number of Hospitalizations in Ohio']
death=dset.loc[len(dset)-1,'Total Deaths']
icu=dset.loc[len(dset)-1, 'Number of ICU Admissions']

x = {
    'Mild': (total-hosp-death-icu)/total*100,
    'Hospital': hosp/total*100,
    'ICU' : icu/total*100,
    'Death': death/total*100   
}

data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(x)]

pie = figure(plot_height=350, title="Breakdown of Infection Severity", toolbar_location=None,
           tools="hover", tooltips="@country: @value", x_range=(-0.5, 1.0))

pie.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='country', source=data)

pie.axis.axis_label=None
pie.axis.visible=False
pie.grid.grid_line_color = None
output_notebook()


output_file("COVID19.html")
plt.show(column(p,p2,pie))

Loading BokehJS ...

In [ ]:
page.content


In [ ]:
statscards

In [ ]:
np.log(dset.loc[4:,'Confirmed Cases'])